In [15]:
import pandas as pd
from pydantic import BaseModel, field_validator
from datetime import datetime
from sqlalchemy import create_engine
from typing import Optional
import json
from pathlib import Path
from datetime import datetime

now = datetime.now()

In [16]:
df = pd.read_parquet("//Users//sdedeoglu//Desktop//python//case_data.parquet.gzip")
#or
#df = pd.read_parquet("case_data.parquet")

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350690 entries, 0 to 350689
Data columns (total 27 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   request_id                    350690 non-null  object        
 1   funnel_id                     350690 non-null  object        
 2   session_id                    350690 non-null  object        
 3   user_id                       350590 non-null  float64       
 4   user_agent                    350690 non-null  object        
 5   device_type                   350690 non-null  object        
 6   ip_address                    350690 non-null  object        
 7   timestamp                     350690 non-null  datetime64[ns]
 8   page_name                     350690 non-null  object        
 9   subscriber_id                 349970 non-null  float64       
 10  has_email_contact_permission  234678 non-null  object        
 11  has_phone_con

In [18]:
#connection
path = Path("//Users//sdedeoglu//Desktop//python//config.json").read_text(encoding="utf-8")
config = json.loads(path) 

kullanici = config['kullanici']
sifre = config['sifre']  
host = config['host']  
port = config['port']  
veritabani = config['veritabani'] 
engine = create_engine(f"mysql+pymysql://{kullanici}:{sifre}@{host}:{port}/{veritabani}")

 #Pydantic 

@field_validator("alan1", "alan2", mode="before") — alan değerleri Pydantic model oluşturulmadan önce bu fonksiyondan geçirilir.
mode="after" — önce Pydantic tipi uygulanır, sonra validator çalışır.
Birden fazla alanı aynı validator ile hedefleyebilirsin.
Döndürülen değer, o alanın nihai değeri olur; hata fırlatılırsa validation hatası oluşur.

In [ ]:
true_set = {"yes", "true", "1", "y", "evet"}
false_set = {"no", "false", "0", "n", "hayir", "hayır"}

formats = [
"%Y-%m-%d %H:%M:%S.%f",  
"%Y-%m-%d %H:%M:%S",     
"%Y-%m-%dT%H:%M:%S.%fZ", 
"%Y-%m-%dT%H:%M:%S",     
"%d/%m/%Y %H:%M:%S",     
"%d-%m-%Y %H:%M:%S"]

status = {
"done": "success",
"ok": "success",
"paid": "success",
"fail": "failed",
"error": "failed"}

In [20]:
class UserModel(BaseModel):
    #veri tiplerinin belirlenmesi
    user_id: Optional[float]
    subscriber_id: Optional[float]
    country: str
    has_email_contact_permission: Optional[bool]
    has_phone_contact_permission: Optional[bool]

    @field_validator("has_email_contact_permission", "has_phone_contact_permission", mode="before")
    def to_bool(cls, v):
        #izin sutunlarinin bool tipine cevrilmesi
        if v is None:
            return None
        if isinstance(v, str):
            s = v.strip().lower()
            if s in true_set:
                return True
            if s in false_set:
                return False
        else:
            return None

In [21]:
class EventModel(BaseModel):
    request_id: str
    session_id: str
    funnel_id: str
    timestamp: datetime
    page_name: str
    search_query: Optional[str]
    destination_id: Optional[float]
    num_guests: Optional[float]
    #timestamp sutununda belirli formatlarda gelen tarih degerlerini datetime objesine cevirme
    @field_validator("timestamp", mode="before")
    def parse_timestamp(cls, v):
        if isinstance(v, datetime):
            return v
        if v is None:
            raise ValueError("timestamp boş olamaz")
        if isinstance(v, str):
            try:
                for fmt in formats:
                    return datetime.strptime(v, fmt)
            except ValueError:
                raise ValueError("timestamp geçersiz")

In [22]:
class HotelModel(BaseModel):
    hotel_id: int
    hotel_price: Optional[float]
    currency: str
#otel ucretlerinde gereksiz karakterlerin temizlenmesi 
    @field_validator("hotel_price", mode="before")
    def clean_price(cls, v):
        if v is None:
            return None
        if isinstance(v, str):
            v = v.replace(",", ".").replace("$", "").strip()
        try:
            return float(v)
        except (ValueError, TypeError):
            return None

In [23]:
class PaymentModel(BaseModel):
    request_id: str
    payment_status: Optional[str]
    confirmation_number: Optional[str]
#odeme sutununun belirlenen fortmata cevrilmesi
    @field_validator("payment_status", mode="before")
    def normalize_status(cls, v):
        if v is None:
            return None
        else:
            v = v.strip().lower()
        return status.get(v, v)
#icerik kontrolu
    @field_validator("payment_status")
    def check_valid_values(cls, v):
        allowed = {"pending", "success", "failed", None}
        if v not in allowed:
            raise ValueError(f"Invalid payment status: {v}")
        return v

In [24]:
class SessionModel(BaseModel):
    session_id: str
    user_id: Optional[float]
    user_agent: Optional[str]
    device_type: Optional[str]
    ip_address: Optional[str]
    utm_source: Optional[str]

    @field_validator("session_id", mode="before")
    def session_id_not_empty(cls, v):
        if v is None:
            raise ValueError("session_id boş olamaz")
        if isinstance(v, str):
            v = v.strip()
            if not v:
                raise ValueError("session_id boş olamaz")
            return v
        return str(v)

In [25]:
#Validation & Normalization

def validate_with_pydantic(df):
    users, sessions, events, hotels, payments = [], [], [], [], []

    for row in df.to_dict(orient="records"):
        try:
            users.append(UserModel(**row).model_dump(exclude_none=True))
            sessions.append(SessionModel(**row).model_dump(exclude_none=True))
            events.append(EventModel(**row).model_dump(exclude_none=True))
            hotels.append(HotelModel(**row).model_dump(exclude_none=True))
            payments.append(PaymentModel(**row).model_dump(exclude_none=True))
        except Exception as e:
            print(f"Validation error: {e}")

    return (
        pd.DataFrame(users).drop_duplicates(subset=['user_id']),
        pd.DataFrame(sessions).drop_duplicates(subset=['session_id']),
        pd.DataFrame(events),
        pd.DataFrame(hotels).drop_duplicates(subset=['hotel_id']),
        pd.DataFrame(payments)
    )

users, sessions, events, hotels, payments = validate_with_pydantic(df)

#ornek hatalar
#Validation error: float() argument must be a string or a number, not 'NoneType'

In [26]:
users["updated_Date"] = now
sessions["updated_Date"] = now
events["updated_Date"] = now
hotels["updated_Date"] = now
payments["updated_Date"] = now

In [28]:
users.to_sql(name='users', con=engine,if_exists='replace',chunksize=10000, index=False)
sessions.to_sql(name='sessions', con=engine,if_exists='replace',chunksize=10000, index=False)
events.to_sql(name='events', con=engine,if_exists='replace',chunksize=10000, index=False)
hotels.to_sql(name='hotels', con=engine,if_exists='replace',chunksize=10000, index=False)
payments.to_sql(name='payments', con=engine,if_exists='replace',chunksize=10000, index=False)

350690

          ┌─────────────────────────────────┐
          │             USERS               │
          │─────────────────────────────────│
          │ user_id (PK) - float            │
          │ subscriber_id - float           │
          │ country - string nullable       │
          │ has_email_contact_permission    │
          │   - bool nullable               │
          │ has_phone_contact_permission    │
          │   - bool nullable               │
          │ updated_Date - datetime         │
          └─────────────┬───────────────────┘
                        │ 1-to-many
                        ▼
          ┌─────────────────────────────────┐
          │            SESSIONS             │
          │─────────────────────────────────│
          │ session_id (PK) - string        │
          │ user_id (FK) - float            │
          │ user_agent - string nullable    │
          │ device_type - string nullable   │
          │ ip_address - string nullable    │
          │ utm_source - string nullable    │
          │ updated_Date - datetime         │
          └─────────────┬───────────────────┘
                        │ 1-to-many
                        ▼
          ┌─────────────────────────────────┐
          │             EVENTS              │
          │─────────────────────────────────│
          │ request_id - string             │
          │ session_id (FK) - string        │
          │ funnel_id - string              │
          │ hotel_id (FK) - int             │
          │ timestamp - datetime            │
          │ page_name - string nullable     │
          │ search_query - string nullable  │
          │ destination_id - float nullable │
          │ num_guests - float nullable     │
          │ updated_Date - datetime         │
          └─────────────┬───────────────────┘
                        │ 1-to-one
                        ▼
          ┌─────────────────────────────────┐
          │            PAYMENTS             │
          │─────────────────────────────────│
          │ request_id (FK) - string        │
          │ payment_status - string         │
          │   ("pending","success","failed")│
          │ confirmation_number - string    │
          │   nullable                      │
          │ updated_Date - datetime         │
          └─────────────────────────────────┘

          ┌─────────────────────────────────┐
          │             HOTELS              │
          │─────────────────────────────────│
          │ hotel_id (PK) - int             │
          │ hotel_price - float nullable    │
          │ currency - string nullable      │
          │ updated_Date - datetime         │
          └─────────────────────────────────┘
                        ▲
                        │ many-to-one
                        │
          ┌─────────────────────────────────┐
          │             EVENTS              │
          │   (hotel_id referansı)          │
          └─────────────────────────────────┘